# **RUNNING SHELL COMMANDS USING PYTHON**

In [1]:
import subprocess
import os

In [2]:
# capture_output=True an argument of line below that has to be used without stdout=subprocess.PIPE
p1 = subprocess.run("dir", shell=True, stdout=subprocess.PIPE, text=True)
#                                               ^ the file where output is redirected, we can use custom txt file or a log file to store this as well

print(p1.stdout)

 Volume in drive D is DATA
 Volume Serial Number is 2C12-72A4

 Directory of d:\WORK\PYTHON\Projects\NOVA_AI\notebooks

30-07-2024  12:40    <DIR>          .
30-07-2024  12:57    <DIR>          ..
11-08-2024  21:24            12,629 experiments.ipynb
30-07-2024  12:40    <DIR>          outputs
               1 File(s)         12,629 bytes
               3 Dir(s)  909,292,494,848 bytes free



In [3]:
with open("outputs/output.txt", 'w') as f:
    p1 = subprocess.run("dir", shell=True, stdout=f, text=True)

We have created an outputs folder and inside that we can see that we have a txt file with the name output.txt storing the result of the command that we just executed

In [4]:
p1 = subprocess.run("dir", shell=True, capture_output=True, text=True)
print(p1.stdout)

 Volume in drive D is DATA
 Volume Serial Number is 2C12-72A4

 Directory of d:\WORK\PYTHON\Projects\NOVA_AI\notebooks

30-07-2024  12:40    <DIR>          .
30-07-2024  12:57    <DIR>          ..
11-08-2024  21:24            12,629 experiments.ipynb
30-07-2024  12:40    <DIR>          outputs
               1 File(s)         12,629 bytes
               3 Dir(s)  909,292,494,848 bytes free



What if the command fails, does python returns any traceback? NO

why? One possible reason is that the error isn't generated by the python script but actually the command line, which just returns a non-zero return code, indicating that execution was unsuccessful

In [5]:
subprocess.run("dire", shell=True, capture_output=True, text=True)

CompletedProcess(args='dire', returncode=1, stdout='', stderr="'dire' is not recognized as an internal or external command,\noperable program or batch file.\n")

see the return code is 1, can we handle this in python??? for sure

In [6]:
p1 = subprocess.run("dire", shell=True, capture_output=True, text=True)

if p1.returncode != 0:
    print("COULDN'T EXECUTE COMMAND")
else:
    print(p1.stdout)

COULDN'T EXECUTE COMMAND


Now we have handled that exception as well, but what is the cause of the error???? we can show that too

In [7]:
p1 = subprocess.run("dire", shell=True, capture_output=True, text=True)

if p1.returncode != 0:
    print("COULDN'T EXECUTE COMMAND")
    print(p1.stderr)
else:
    print(p1.stdout)

COULDN'T EXECUTE COMMAND
'dire' is not recognized as an internal or external command,
operable program or batch file.



Now we have returned the error cause as well

But can we make python handle this error and throw an inbuilt traceback???
for sure

In [8]:
p1 = subprocess.run("dire", shell=True, capture_output=True, text=True, check=True)

CalledProcessError: Command 'dire' returned non-zero exit status 1.

Now python handled the exception and threw a CalledProcessError

But what if we want to store the error in a log file? or ignore the error

In [9]:
p1 = subprocess.run("dire", shell=True, stderr=subprocess.DEVNULL) #ignore the error and returns None
print(p1.stdout)

None


Or we can store it in a log file

In [10]:
#p1 = subprocess.run("dire", shell=True, stderr="""file object here""")

## **CHANGING BRIGHTNESS OF THE DISPLAY USING CMD PROMPT**

In [12]:
p1 = subprocess.run("(Get-WmiObject -Namespace root/WMI -Class WmiMonitorBrightnessMethods).WmiSetBrightness(1,75)", shell=True, text=True)

print(p1.returncode)

1


Unable to execute here but this command works fine in powershell

In [14]:
p1 = subprocess.run(["powershell", "(Get-WmiObject -Namespace root/WMI -Class WmiMonitorBrightnessMethods).WmiSetBrightness(1,50)"], capture_output=True, shell=True, text=True)

Now it's working just fine and we can change the brightness using a python script as well

## **CHANGING VOLUME AND MUTE UNMUTE USING PYTHON SCRIPT**

PROBLEMS:
1. There is no single command like the one above, to control the volume of the system
2. One solution is install an external api and then use it functions to control the volume
3. But that would mean in every system where try to execute the ai, it would need that api 
4. So this solution is discarded
5. Another solution is that using powershell we can give the command that the computer gets when we press the volume up or down or toggle mute buttons
6. We'd be needed to find a solution to a problem so that a user can just say loud or louder to increase the volume or give a specific percentage of loudness

The required powershell commands:
1. Volume up : (new-object -com wscript.shell).SendKeys([char]174)
2. Volume down : (new-object -com wscript.shell).SendKeys([char]175)
3. Toggle mute : (new-object -com wscript.shell).SendKeys([char]173)

### Current Updates:    11/Aug/2012
1. So far scouring the internet, it sounds impossible to get the current volume levels.
2. By executing the above command we are just passing the characters window recieves when we manually press the volume buttons
3. But that's not the case with brightness, we can actually get the current level of brightness and then change it accordingly
4. That too with python scripts

## **GETTING THE BRIGHTNESS VALUES AND CHANGING THEM AS PER OUR NEEDS**

In [37]:
p1 = subprocess.run(["powershell", "Get-Ciminstance -Namespace root/WMI -ClassName WmiMonitorBrightness"], capture_output=True, shell=True, text=True)
current_brightness = p1.stdout[47:49]

print("Current Brightness : ", current_brightness)

Current Brightness :  49


We can hardcode this because it is going to follow the same structure across the windows devices

and now we can manually change the brightness levels using python script

In [42]:
p1 = subprocess.run(["powershell", "Get-Ciminstance -Namespace root/WMI -ClassName WmiMonitorBrightness"], capture_output=True, shell=True, text=True)
current_brightness = p1.stdout[47:49]
print("Current Brightness : ", current_brightness)

x = int(input("Enter the brightness value you want to go to : "))

p1 = subprocess.run(["powershell", f"(Get-WmiObject -Namespace root/WMI -Class WmiMonitorBrightnessMethods).WmiSetBrightness(1,{x})"], capture_output=True, shell=True, text=True)
p1 = subprocess.run(["powershell", "Get-Ciminstance -Namespace root/WMI -ClassName WmiMonitorBrightness"], capture_output=True, shell=True, text=True)
current_brightness = p1.stdout[47:49]
print("Current Brightness : ", current_brightness)

Current Brightness :  10
Current Brightness :  50
